# Evaluation of KeyBERTSentimentAware

### **Keyword Extraction Model Evaluation on Movie Reviews**

This notebook evaluates and compares different keyword extraction models applied to movie reviews, with a specific focus on assessing how well each model captures not only **semantic relevance** but also **sentiment awareness** in its keyword rankings.

We assess the performance of two models:

- **Base** KeyBERT model  
- **Sentiment-aware** extension (`KeyBERTSentimentAware`), which integrates a custom sentiment classifier to adjust keyword relevance scores based on predicted sentiment

This notebook adopts a **global evaluation approach**: all predicted and ground truth keywords across the entire dataset are aggregated by movie **before** computing each metric. This provides a **holistic view** of each model's performance, unaffected by review-level variance.

We use a set of annotated ground truth keywords per movie (from IMDb), and the top-5 predicted keywords (with scores) per review for both models.

### **Evaluation Layers**

#### 1. **Basic (Unweighted) Metrics**

- **Precision**, **Recall**, and **F1-score** computed globally via approximate binary matching.
- A predicted keyword is correct if it approximately matches any of the movie’s ground truth keywords.
- All keywords from all reviews are flattened and compared in aggregate.

#### 2. **Score-Aware Metrics**

- **Weighted Precision**, **Recall**, and **Weighted F1-score**:
  - Each predicted keyword is weighted by its score.
  - Correct predictions contribute proportionally to their confidence.
- **nDCG@5 (Normalized Discounted Cumulative Gain)**:
  - Evaluates whether correct keywords are ranked near the top globally.
  - Relevance is discounted by position, rewarding better keyword orderings.

#### 3. **Semantic Evaluation (Embedding-Based)**

- All predicted and ground truth keywords are embedded using a **sentence-transformer** model.
- **Cosine similarity** is used to detect approximate **semantic matches**.
- A predicted keyword is correct if its similarity with any ground truth keyword exceeds a given threshold (e.g., 0.75).
- **Semantic Precision**, **Recall**, and **F1-score** are computed globally based on these soft matches.

#### 4. **Sentiment-Aware Evaluation**

These metrics assess whether the predicted keywords reflect the **emotional tone** of the content, not just its semantic meaning.

- **Average Sentiment Similarity (ASS)**  
  Measures how emotionally aligned the predicted keywords are with a reference sentiment value (e.g., average sentiment of the ground truth).  
  Each review’s predicted keywords are converted into a sentiment vector and compared to a fixed sentiment reference using **cosine similarity**.  
  The final value is the average similarity across all reviews, clamped to the [0, 1] range.  
  High values indicate strong emotional coherence with the global reference.

- **Review-wise Sampled Sentiment Similarity (RSSS)**  
  Introduces per-review robustness by comparing sampled predicted keywords with sampled ground truth keywords.  
  For each review, multiple random samples of predicted keywords are compared to the same ground truth sentiment vector.  
  The final score is the average of all these similarities, capturing local variations in emotional tone and offering **granular insight** into sentiment alignment.

### **Why Sentiment-Aware Evaluation Matters**

The base KeyBERT model selects keywords purely based on semantic relevance, while the sentiment-aware version ranks keywords by combining semantic and emotional signals.

Traditional evaluation metrics may overlook whether the extracted keywords reflect the emotional tone of a review. The Sentiment Rank Correlation (SRC) addresses this gap by explicitly measuring the relationship between emotional expressiveness and keyword relevance.

This multi-dimensional evaluation provides a robust framework for comparing keyword extraction models, revealing differences in both informational content and emotional alignment.


## Setup: Installing and Importing Required Libraries

In [1]:
import subprocess
import sys

# List of required packages
required_packages = {
    "pandas", "numpy", "tqdm", "transformers", "torch", "scipy", "nltk"
}

def install_package(package):
    """Installs a package using pip if it's not already installed."""
    try:
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check and install missing packages
for package in required_packages:
    install_package(package)


pandas is already installed.
numpy is already installed.


/Users/manuelemustari/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/manuelemustari/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformers is already installed.
torch is already installed.
scipy is already installed.
nltk is already installed.
tqdm is already installed.


In [2]:
# Standard Library
import os      # Operating system interfaces (e.g., file and directory handling)
import re      # Regular expressions for advanced pattern matching in text
import math    # Mathematical functions (e.g., exp, log, sqrt)
import random  # Random sampling utilities (e.g., for bootstrapping or sampling keywords)

# Data Handling and Computation
import pandas as pd      # Data manipulation and analysis (DataFrames)
import numpy as np       # Numerical operations and linear algebra (arrays, vector math)

# Progress Monitoring
from tqdm import tqdm    # Progress bars for loops and data processing

# Machine Learning & Transformers
import torch                                       # Core PyTorch tensor operations
import torch.nn.functional as F                   # Functional API for layers, activations, loss functions
from transformers import AutoTokenizer, AutoModel  # Hugging Face: load pre-trained transformer models and tokenizers

# Evaluation and Statistics
from sklearn.metrics.pairwise import cosine_similarity  # Vector similarity computation (e.g., for sentiment vectors)
from scipy.stats import spearmanr                       # Spearman rank correlation for monotonic relationship analysis

# Sentiment Analysis
import nltk                                              # Natural Language Toolkit (used to load VADER lexicon)
from nltk.sentiment import SentimentIntensityAnalyzer   # VADER sentiment analyzer (suitable for short texts and keywords)
nltk.download("vader_lexicon")                           # Ensure the VADER lexicon is available locally



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/manuelemustari/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## Load Available Movies from Dataset

This section lists all the available movies stored as `.pkl` files inside the review dataset directory.

- It defines the root path (`../Dataset/Reviews_By_Movie`) where all review files are saved.
- It automatically detects and lists all movie filenames (removing the `.pkl` extension).

In [3]:
# Define root directory
root_dir = "../Dataset/Reviews_By_Movie"

# List all available movies
available_movies = sorted([f[:-4] for f in os.listdir(root_dir) if f.endswith(".pkl")])
print("Available movies:", available_movies)

Available movies: ['GoodBadUgly', 'HarryPotter', 'IndianaJones', 'LaLaLand', 'Oppenheimer', 'Parasite', 'SW_Episode1', 'SW_Episode2', 'SW_Episode3', 'SW_Episode4', 'SW_Episode5', 'SW_Episode6', 'SW_Episode7', 'SW_Episode8', 'SW_Episode9']


## Select a Movie and Load its Ground Truth Keywords

In this step, we load the keyword extraction results for a specific movie and retrieve the corresponding ground truth keywords. The goal is to use these annotated keywords for evaluation and comparison with automatically extracted ones.

In [22]:
# Set the name of the movie to be evaluated
movie_name = "IndianaJones"

# Load the extracted keywords for the selected movie from a pickle file
# The file path is dynamically built using the movie name
selected_film = pd.read_pickle(f"../Dataset/Extracted_Keywords/kw_{movie_name}.pkl")

# Retrieve the Movie_ID of the selected film
# Assumes that the file contains a DataFrame with at least one row
selected_film_id = selected_film["Movie_ID"].iloc[0]

# Load the full dataset containing the ground truth keywords
# for all movies in the evaluation set
keywords = pd.read_pickle("../Dataset/keywords_ground_truth.pkl")

# Filter the ground truth dataset to extract only the keywords for the selected movie
kw_ground_truth = keywords[keywords["Movie_ID"] == selected_film_id]

## Keyword Matching and Evaluation Functions (Basic – Unweighted)

This block defines the core utility functions used to evaluate predicted keywords against the ground truth. These functions perform a **binary, unweighted evaluation**, ignoring confidence scores and ranking information.

The evaluation pipeline includes the following steps:

- **Normalization**: all keywords are lowercased, stripped of punctuation, and cleaned of extra whitespace to ensure consistent text matching.

- **Approximate Matching**: a relaxed rule considers two keywords as a match if:
  - They are exactly equal (after normalization), or
  - One is a substring of the other (e.g., *"social satire"* is considered a match with *"satire"*).

- **Global Evaluation**: for each model, all keywords predicted across the reviews of a given movie are aggregated, and then compared to the global set of ground truth keywords for that movie.

- **Metrics**: we compute **Precision**, **Recall**, and **F1-score** based on the number of approximate matches between the predicted and ground truth keywords.

In [5]:
def normalize_kw(kw):
    """
    Normalize a keyword string by:
    - Converting to lowercase
    - Removing punctuation and non-alphanumeric characters (except spaces)
    - Stripping leading and trailing whitespace

    Args:
        kw (str): The keyword string to normalize.

    Returns:
        str: The normalized keyword.
    """
    kw = kw.lower()
    kw = re.sub(r"[^a-zA-Z0-9\s]", "", kw)  # Keep only alphanumeric characters and whitespace
    return kw.strip()


def is_approx_match(kw, gt_keywords):
    """
    Check if a predicted keyword approximately matches any ground truth keyword.

    A match is considered approximate if:
    - The predicted keyword is exactly equal to a ground truth keyword
    - OR the predicted keyword is a substring of a ground truth keyword
    - OR a ground truth keyword is a substring of the predicted one

    Args:
        kw (str): The normalized predicted keyword.
        gt_keywords (List[str]): A list of normalized ground truth keywords.

    Returns:
        bool: True if an approximate match is found, False otherwise.
    """
    for gt in gt_keywords:
        if kw == gt or kw in gt or gt in kw:
            return True
    return False


def evaluate_keywords(all_pred_keywords, all_gt_keywords):
    """
    Evaluate global precision, recall, and F1-score across a dataset using approximate matching.

    This function compares predicted keywords to ground truth keywords for each review.
    Matching is performed using approximate string comparison, and each ground truth keyword
    can be matched only once to ensure fairness. The metrics are aggregated globally,
    not per-review.

    Args:
        all_pred_keywords (List[List[str]]): 
            A list where each element is a list of predicted keywords for a single review.
        all_gt_keywords (List[List[str]]): 
            A list where each element is a list of ground truth keywords for the corresponding review.

    Returns:
        Tuple[float, float, float]: Global precision, recall, and F1-score based on approximate matching.
    """
    global_match_count = 0     # Total number of matched keywords across all reviews
    global_pred_count = 0      # Total number of predicted keywords
    global_gt_count = 0        # Total number of ground truth keywords

    # Iterate through each review's predictions and ground truths
    for pred_keywords, gt_keywords in zip(all_pred_keywords, all_gt_keywords):
        # Normalize and sort keywords to ensure consistent behavior
        pred_keywords = sorted([normalize_kw(k) for k in pred_keywords])
        gt_keywords = sorted([normalize_kw(k) for k in gt_keywords])

        global_pred_count += len(pred_keywords)
        global_gt_count += len(gt_keywords)

        matched_gts = set()  # Track which ground truth keywords have already been matched

        for pred in pred_keywords:
            for gt in gt_keywords:
                if gt not in matched_gts and is_approx_match(pred, [gt]):
                    global_match_count += 1
                    matched_gts.add(gt)  # Avoid matching the same GT keyword multiple times
                    break

    # Compute global metrics
    precision = global_match_count / global_pred_count if global_pred_count else 0
    recall = global_match_count / global_gt_count if global_gt_count else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    return precision, recall, f1

### Evaluate and Compare Models on Keyword Extraction (Basic – Unweighted)

This section evaluates two keyword extraction models — **base** and **sentiment-enhanced** — against the ground truth annotations.

For each model, we collect all predicted keywords across all reviews in the selected movie and compare them to the ground truth keywords using **binary approximate matching**.

The evaluation computes **global precision, recall, and F1-score**, considering the entire set of predictions and ground truth keywords as a whole.

In [6]:
# Define the models to be evaluated
models_to_evaluate = ["base", "sentiment"]

# Extract the list of ground truth keywords for the selected movie
ground_truth_keywords = [normalize_kw(kw) for kw in kw_ground_truth["Keyword"].tolist()]

# Dictionary to store all predicted keywords per model (across all reviews)
all_predictions = {model: [] for model in models_to_evaluate}

# Iterate over each review in the selected film's predictions
for _, row in selected_film.iterrows():
    for model in models_to_evaluate:
        pred_col = f"keywords_{model}"

        if pred_col in row and isinstance(row[pred_col], list):
            predicted_keywords = [
                normalize_kw(kw) for kw, _ in row[pred_col] if isinstance(kw, str)
            ]
            
            # Remove duplicates per review
            seen = set()
            unique_kw = [kw for kw in predicted_keywords if kw not in seen and not seen.add(kw)]

            all_predictions[model].append(unique_kw)

# Evaluate each model globally
summary = {}
for model in models_to_evaluate:
    precision, recall, f1 = evaluate_keywords(
        all_predictions[model],  # List of lists
        ground_truth_keywords
    )

    summary[model] = {
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1-score": round(f1, 4)
    }

# Convert and display
summary_df = pd.DataFrame(summary).T
summary_df.columns = ["Precision", "Recall", "F1-score"]
summary_df.style.format(precision=4).set_caption("Global Evaluation Summary")


,Precision,Recall,F1-score
base,0.9125,0.3685,0.5250
sentiment,0.8805,0.3391,0.4896


## Score-Aware Evaluation: Weighted Metrics

This extended evaluation considers the **confidence scores** assigned by the model to each predicted keyword, allowing us to measure not only whether the predictions are correct but also how confidently and effectively they are ranked.

#### Score-Aware Metrics

- **Weighted Precision**: Reflects the proportion of the model’s total confidence assigned to correct keywords. High confidence in incorrect predictions lowers this score.

- **Weighted Recall**: Measures how much of the ground truth is recovered, weighted by the confidence of correct predictions.

- **Weighted F1-score**: The harmonic mean of weighted precision and recall, balancing accuracy with coverage.

In [7]:
def normalize_kw(kw):
    """
    Normalize a keyword string by:
    - Converting to lowercase
    - Removing punctuation and non-alphanumeric characters (except spaces)
    - Stripping leading and trailing whitespace

    Args:
        kw (str): The keyword string to normalize.

    Returns:
        str: The normalized keyword.
    """
    kw = kw.lower()
    kw = re.sub(r"[^a-zA-Z0-9\s]", "", kw)  # Keep only alphanumeric characters and whitespace
    return kw.strip()


def is_approx_match(kw, gt_keywords):
    """
    Check if a predicted keyword approximately matches any ground truth keyword.

    A match is considered approximate if:
    - The predicted keyword is exactly equal to a ground truth keyword
    - OR the predicted keyword is a substring of a ground truth keyword
    - OR a ground truth keyword is a substring of the predicted one

    Args:
        kw (str): The normalized predicted keyword.
        gt_keywords (List[str]): A list of normalized ground truth keywords.

    Returns:
        bool: True if an approximate match is found, False otherwise.
    """
    for gt in gt_keywords:
        if kw == gt or kw in gt or gt in kw:
            return True
    return False


def score_gt_keywords_from_rank(gt_keywords):
    return [
        (kw, 1 / math.log2(i + 2)) for i, kw in enumerate(gt_keywords)
    ]

def evaluate_keywords_weighted(all_predicted_kw_score, all_gt_keywords):
    """
    Evaluate global weighted precision, recall, and F1-score across multiple reviews.
    Matching is performed using approximate matching. Each predicted keyword contributes
    to the precision proportionally to its confidence score. Each ground truth keyword contributes
    to recall proportionally to its rank-based score.

    Args:
        all_predicted_kw_score (List[List[Tuple[str, float]]]): 
            A list of predicted keyword-score pairs per review.
        all_gt_keywords (List[List[str]]): 
            A list of ground truth keyword lists per review.

    Returns:
        Tuple[float, float, float]: Weighted precision, recall, and F1-score.
    """
    total_pred_score = 0.0  # Sum of all predicted keyword scores
    matched_pred_score = 0.0  # Sum of scores of correctly predicted keywords
    total_gt_score = 0.0  # Sum of all ground truth scores
    matched_gt_score = 0.0  # Sum of scores of matched ground truth keywords

    for pred_kw_score, gt_kw in zip(all_predicted_kw_score, all_gt_keywords):
        # Normalize and score ground truth keywords
        gt_kw_scored = score_gt_keywords_from_rank([normalize_kw(k) for k in gt_kw])
        pred_kw_score = [
            (normalize_kw(kw), score) for kw, score in pred_kw_score if isinstance(kw, str)
        ]

        total_pred_score += sum(score for _, score in pred_kw_score)
        total_gt_score += sum(score for _, score in gt_kw_scored)

        matched_gts = set()

        for kw, score in pred_kw_score:
            for gt_kw, gt_score in gt_kw_scored:
                if gt_kw not in matched_gts and is_approx_match(kw, [gt_kw]):
                    matched_pred_score += score
                    matched_gt_score += gt_score
                    matched_gts.add(gt_kw)
                    break

    precision = matched_pred_score / total_pred_score if total_pred_score > 0 else 0
    recall = matched_gt_score / total_gt_score if total_gt_score > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

### Evaluate and Compare Models on Keyword Extraction (Weighted)

In this section, we evaluate the overall performance of each model using **score-aware metrics** computed **globally across all reviews**:

- **Weighted Precision, Recall, and F1-score**: These metrics incorporate the **confidence scores** assigned to each predicted keyword, reflecting how much of the model’s confidence is placed on correct predictions.

This global evaluation provides a holistic view of each model’s effectiveness in ranking and selecting relevant keywords across the entire dataset.

In [8]:
# Models to evaluate
models_to_evaluate = ["base", "sentiment"]

# Ground truth keywords (same for all reviews in the selected film)
ground_truth_keywords = kw_ground_truth["Keyword"].tolist()

# Prepare data structures to hold predictions for each model
all_predicted_kw_score = {model: [] for model in models_to_evaluate}

# Collect predictions and GT for each review
for _, row in selected_film.iterrows():
    for model in models_to_evaluate:
        pred_col = f"keywords_{model}"

        # Skip if no prediction or wrong format
        if pred_col in row and isinstance(row[pred_col], list):
            predicted_kw_score = [(kw, score) for kw, score in row[pred_col] if isinstance(kw, str)]
            
            # Remove duplicates per review
            seen = set()
            unique_pred = [(kw, score) for kw, score in predicted_kw_score if kw not in seen and not seen.add(kw)]
            all_predicted_kw_score[model].append(unique_pred)

# Dictionary to store global evaluation results
weighted_summary = {}

# Evaluate each model globally
for model in models_to_evaluate:
    preds = all_predicted_kw_score[model]

    # Global weighted metrics
    w_precision, w_recall, w_f1 = evaluate_keywords_weighted(preds, ground_truth_keywords)

    # Store results
    weighted_summary[model] = {
        "weighted_precision": round(w_precision, 4),
        "weighted_recall": round(w_recall, 4),
        "weighted_f1": round(w_f1, 4),
    }

# Convert summary to DataFrame
summary_df = pd.DataFrame(weighted_summary).T  # Models as rows

# Rename columns
summary_df.columns = [
    "Weighted Precision",
    "Weighted Recall",
    "Weighted F1-score",
]

# Display final table
summary_df.style.format(precision=4).set_caption("Global Score-Aware Evaluation Summary")


,Weighted Precision,Weighted Recall,Weighted F1-score
base,0.9176,0.4902,0.6390
sentiment,0.8955,0.4489,0.5981


## Semantic Evaluation (Base vs Sentiment)

In this section, we evaluate and compare the **Base** and **Sentiment-enhanced** keyword extraction models using a **semantic similarity approach** based on contextual embeddings.

Traditional evaluation metrics rely on exact or approximate string matching between predicted and ground truth keywords. However, this approach may miss semantically related terms that differ lexically but convey the same meaning — such as *"scam"* and *"fraud"*.

To address this limitation, we adopt a **global semantic evaluation**, where all predicted and ground truth keywords across the dataset are compared using **dense sentence embeddings** generated by a pre-trained transformer (e.g., Sentence-BERT).

#### **Semantic Evaluation Procedure**

1. **Embedding Keywords Globally**  
   All predicted and ground truth keywords across all reviews are embedded into high-dimensional vectors using the same transformer model. Ground truth keywords are embedded **once**, and all vectors are normalized to allow cosine similarity comparisons.

2. **Computing Similarity Matrix**  
   For each model, we compute a cosine similarity matrix between **all predicted keywords** and **all ground truth keywords**.

3. **Matching Threshold**  
   A predicted keyword is considered a **semantic match** if its cosine similarity with at least one ground truth keyword exceeds a fixed threshold (e.g., **0.65**). This allows for flexible yet meaningful semantic alignment.

4. **Global Semantic Precision**  
   The proportion of predicted keywords that have at least one semantic match in the ground truth. This reflects how many of the model's predictions are semantically relevant.

5. **Global Semantic Recall**  
   The proportion of ground truth keywords that are captured by semantically similar predictions. This indicates how well the model covers the key concepts.

6. **Global Semantic F1-score**  
   The harmonic mean of semantic precision and recall, summarizing both relevance and coverage into a single score.

This evaluation:

- Is **more robust** than string-based metrics.
- **Captures meaning**, not just surface forms.
- Helps evaluate models that paraphrase or generalize beyond exact matches.

This evaluation complements previous metrics and provides a more **realistic estimate** of how well the models capture the essence of user-annotated keywords in a global and context-aware manner.

In [9]:
# Load a sentence embedding model from the SentenceTransformers family
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model to generate contextual embeddings
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
encoder = AutoModel.from_pretrained(MODEL_NAME).to(device)

# Simple normalization function for keywords:
# - Converts to lowercase
# - Removes punctuation
# - Strips leading/trailing spaces
def normalize_kw(kw):
    kw = kw.lower()
    kw = re.sub(r"[^a-zA-Z0-9\s]", "", kw)  # Keep only alphanumerics and whitespace
    return kw.strip()

def embed_keywords(keywords, device="cuda"):
    """
    Compute sentence embeddings for a list of keyword strings.

    Parameters:
    ----------
    keywords : List[str]
        A list of keyword strings to encode.
    device : str
        Device to run the model on ('cuda' or 'cpu').

    Returns:
    -------
    torch.Tensor
        Normalized embeddings tensor of shape (num_keywords, embedding_dim).
    """
    # Return empty tensor if input list is empty
    if not keywords:
        return torch.empty(0, encoder.config.hidden_size).to(device)

    # Tokenize and prepare inputs for the model
    inputs = tokenizer(keywords, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        # Forward pass through the encoder to get hidden states
        outputs = encoder(**inputs)

        # Use mean pooling on the last hidden state to get fixed-size embeddings
        embeddings = outputs.last_hidden_state.mean(dim=1)

        # Normalize embeddings to unit length for cosine similarity computations
        embeddings = F.normalize(embeddings, p=2, dim=1)

    return embeddings

def evaluate_semantic_keywords(all_pred_keywords, gt_keywords, threshold=0.65, device="cuda"):
    """
    Compute global semantic precision, recall, and F1 score between all predicted keywords
    and ground truth keywords using cosine similarity over embeddings.

    Parameters:
    ----------
    all_pred_keywords : List[List[str]]
        List of predicted keywords for each review.
    gt_keywords : List[str]
        Global list of ground truth keywords for the movie.
    threshold : float
        Cosine similarity threshold for considering a match.
    device : str
        Device to run the model on ('cuda' or 'cpu').

    Returns:
    -------
    precision : float
    recall : float
    f1 : float
    """
    # Early return if either set is empty
    if len(all_pred_keywords) == 0 or len(gt_keywords) == 0:
        return 0.0, 0.0, 0.0

    # Compute embeddings
    pred_emb = embed_keywords(all_pred_keywords, device=device)
    gt_emb = embed_keywords(gt_keywords, device=device)

    # Compute similarity matrix
    sims = torch.matmul(pred_emb, gt_emb.T)

    # Match counting based on threshold
    pred_matches = (sims > threshold).any(dim=1).float().sum().item()
    gt_matches = (sims > threshold).any(dim=0).float().sum().item()

    precision = pred_matches / len(all_pred_keywords)
    recall = gt_matches / len(gt_keywords)
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

    return precision, recall, f1

### Semantic Evaluation of Base and Sentiment Models Using Sentence Embeddings

In this step, we evaluate the **semantic similarity** between the predicted keywords of two models — **Base** and **Sentiment-enhanced** — and the ground truth keywords using **sentence embeddings**.

Unlike exact or approximate string matching, this method leverages **contextual embeddings** from a pre-trained transformer to assess how semantically close the predicted keywords are to the reference keywords.

The evaluation procedure is as follows:

- We extract only the **text** of the predicted keywords for each model, discarding their confidence scores.
- We embed all **predicted** and **ground truth** keywords using the same sentence transformer model.
- Embeddings are **normalized** to ensure cosine similarity is a valid similarity measure.
- For each predicted keyword, we compute the **cosine similarity** with all ground truth keywords.
- A predicted keyword is considered a **semantic match** if its similarity with any ground truth keyword exceeds a fixed threshold (e.g., **0.65**).

Once all matches are determined across all reviews of the selected movie, we compute:

- **Semantic Precision**: Fraction of all predicted keywords (global) that have a semantic match.
- **Semantic Recall**: Fraction of all ground truth keywords that are matched by at least one semantically similar predicted keyword.
- **Semantic F1-score**: Harmonic mean of semantic precision and recall.

This global semantic evaluation better reflects the models’ ability to capture **meaningful and relevant keywords**, even when the wording differs from the ground truth.


In [10]:
# Precompute embeddings for the ground truth keywords once per selected movie
gt_keywords = kw_ground_truth["Keyword"].tolist()

# Define the models to evaluate
models_to_evaluate = ["base", "sentiment"]

# Dictionary to collect all predicted keywords per model (without duplicates)
all_predictions = {model: set() for model in models_to_evaluate}

# Collect predicted keywords across all reviews (as a set for uniqueness)
for _, row in selected_film.iterrows():
    for model in models_to_evaluate:
        pred_col = f"keywords_{model}"

        if pred_col in row and isinstance(row[pred_col], list):
            # Extract keyword strings and normalize
            pred_kw = [normalize_kw(kw) for kw, _ in row[pred_col] if isinstance(kw, str)]
            all_predictions[model].update(pred_kw)  # Add to set (no duplicates)

# Compute semantic evaluation globally for each model
semantic_scores = []
for model in models_to_evaluate:
    pred_kw = list(all_predictions[model])  # Convert back to list
    precision, recall, f1 = evaluate_semantic_keywords(pred_kw, gt_keywords, device=device, threshold=0.65)

    semantic_scores.append({
        "Model": model,
        "Semantic_Precision": round(precision, 4),
        "Semantic_Recall": round(recall, 4),
        "Semantic_F1": round(f1, 4)
    })

# Convert to DataFrame and format
summary_df = pd.DataFrame(semantic_scores).set_index("Model")
summary_df.style.format(precision=4).set_caption("Global Semantic-Aware Evaluation Summary")


,Semantic_Precision,Semantic_Recall,Semantic_F1
Model,,,
base,0.9628,0.7978,0.8726
sentiment,0.9616,0.8162,0.8830


## Sentiment-Aware Evaluation of Keyword Extraction

In this section, we compare **KeyBERTSentimentAware**, our sentiment-enhanced extension of KeyBERT, with the **original KeyBERT** model from a **sentiment perspective**.  
While traditional keyword extraction focuses on semantic relevance, here we aim to evaluate whether the extracted keywords also reflect the **emotional tone** of the content they are drawn from. 

To address this, we introduce two sentiment-based evaluation metrics that assess how emotionally aligned the predicted keywords are with the sentiment expressed in the original content.

#### 1. Average Sentiment Similarity (ASS)  

This metric estimates how close the **overall emotional tone** of the predicted keywords is to a reference value derived from the ground truth. It reflects:

*“On average, how emotionally aligned are the extracted keywords with the ground truth?”*

**How it works**

- Each review has a set of predicted keywords.
- Each keyword is converted into a sentiment score using a predefined formula:  
  `sentiment = 1.0 * pos + 0.5 * neu + 0.0 * neg`
- The resulting sentiment vector is compared to a reference vector of the same length, where each element corresponds to the same ground-truth sentiment value.
- The similarity between the two vectors is computed using **cosine similarity**, which measures how directionally aligned the vectors are in sentiment space.
- The similarity score for each review is **clamped between 0 and 1** to ensure numerical stability and interpretability as a bounded alignment measure.
- Finally, the average similarity is computed **across all reviews**, resulting in a single global score.

**Interpretation**

- Values close to 1 indicate that the predicted keywords align emotionally with the ground truth.
- Values close to 0 suggest an emotional mismatch, even if keywords are semantically reasonable.

This metric provides a **global view of emotional alignment**, making it effective when the model captures overall sentiment well. However, since it relies on a uniform reference vector and aggregates all reviews into a single score, it may overlook variability across individual cases or be affected by outlier behavior from neutral terms.

#### 2. Review-wise Sampled Sentiment Similarity (RSSS)  

To introduce greater robustness and sensitivity to local fluctuations, we extend the previous approach with a **sampled, per-review variant**. This version emphasizes individual reviews and mitigates the effect of biased or skewed sentiment vectors.

**How it works**

For each review:
- Five ground truth keywords are randomly sampled and converted into a sentiment vector using the same scoring formula.
- Five predicted keywords are also randomly sampled (this process is repeated multiple times per review).
- The cosine similarity is computed between the sampled ground truth and prediction vectors.
- Each similarity score is **clamped between 0 and 1** to ensure comparability and avoid values outside the interpretable range due to floating-point errors.
- The average similarity for that review is computed across all sampling iterations.
- In the end, the final score is the **average of all per-review scores**.

This method increases **granularity and robustness**, by accounting for the diversity and variability of both the predicted and ground truth keywords at the review level. Unlike the global method, it captures cases where some predictions are strongly aligned emotionally while others are not.

By averaging over multiple random samples and reviews, this metric produces a **more stable and fair estimate** of emotional alignment, especially in scenarios where keyword sets are heterogeneous.

#### Importance for our work

Our work centers on **topic modeling applied to movie reviews**, where both semantic content and emotional tone are central to user interpretation. Traditional evaluations focus solely on lexical or semantic overlap. However, films are reviewed not just for what they’re about, but for **how they make people feel**. Incorporating sentiment-aware evaluations helps us assess whether a keyword extraction model highlights emotionally charged or affectively neutral aspects of the content. 

These metrics therefore provide an essential layer of analysis to ensure that the extracted keywords are not only topically relevant, but also **emotionally faithful** to the underlying user opinion — a critical factor in domains like entertainment, recommendation systems, and opinion mining.

In [15]:
def compute_average_sentiment_similarity(all_sentiments_per_review, gt_sentiment_value):
    """
    Computes the average cosine similarity between each sentiment vector in a list of reviews
    and a ground truth sentiment value vector.

    Each sentiment vector is compared to a vector of the same length filled with the ground truth value.
    Similarity values are clamped to [0.0, 1.0] before averaging.

    Parameters:
        all_sentiments_per_review (list of list of float): List of sentiment score sequences per review.
        gt_sentiment_value (float): The scalar sentiment reference value (typically derived from ground truth keywords).

    Returns:
        float or None: The average cosine similarity, or None if no valid sentiment vectors are found.
    """

    # Convert each sentiment list into a row vector (1 x n), only if it has more than one element
    vectors = [np.array(s).reshape(1, -1) for s in all_sentiments_per_review if len(s) > 1]

    # If no valid vectors are present, return None
    if not vectors:
        return None

    # Compute cosine similarity between each sentiment vector and a reference vector
    # The reference vector is composed of the same value (gt_sentiment_value) repeated
    similarities = [
        cosine_similarity(v, [[gt_sentiment_value] * v.shape[1]])[0][0]
        for v in vectors
    ]

    # Clamp similarities to be within [0, 1] range to ensure valid similarity values
    similarities = [max(0.0, min(1.0, s)) for s in similarities]

    # Return the average similarity
    return np.mean(similarities)


def compute_reviewwise_sentiment_similarity(df, gt_keywords, pred_col, analyzer, k_samples=20, seed=42):
    """
    Computes the average cosine similarity between sentiment vectors of predicted keywords 
    and a fixed sentiment vector of 5 sampled ground truth keywords, for each review.

    For each review, k_samples sentiment vectors are computed by randomly sampling 5 predicted keywords. 
    These vectors are compared to the ground truth sentiment vector using cosine similarity.

    Parameters:
        df (pd.DataFrame): DataFrame containing reviews and predicted keywords.
        gt_keywords (list of str): List of ground truth keywords.
        pred_col (str): Column name containing predicted keywords as (keyword, score) tuples.
        analyzer (object): Sentiment analyzer with a `.polarity_scores(text)` method.
        k_samples (int): Number of random samples of predicted keywords per review.
        seed (int): Random seed for reproducibility.

    Returns:
        float or None: Average cosine similarity across all reviews, or None if not computable.
    """
    # Create independent random generator for deterministic sampling
    rng = random.Random(seed)

    # Filter valid ground truth keywords
    valid_gt = [kw for kw in gt_keywords if isinstance(kw, str)]

    # If not enough ground truth keywords to sample 5, exit early
    if len(valid_gt) < 5:
        return None

    # Sample 5 ground truth keywords and compute their sentiment vector
    sampled_gt = rng.sample(valid_gt, 5)
    gt_vec = [
        1.0 * analyzer.polarity_scores(kw)["pos"] + 0.5 * analyzer.polarity_scores(kw)["neu"]
        for kw in sampled_gt
    ]
    gt_vec = np.array(gt_vec).reshape(1, -1)

    similarities = []

    # Iterate over each review in the DataFrame
    for _, row in df.iterrows():
        if pred_col not in row or not isinstance(row[pred_col], list):
            continue

        # Extract only valid string keywords
        pred_keywords = [kw for kw, _ in row[pred_col] if isinstance(kw, str)]
        if len(pred_keywords) < 5:
            continue

        review_sims = []

        for _ in range(k_samples):
            # Sample 5 predicted keywords using the same RNG
            sampled_pred = rng.sample(pred_keywords, 5)

            # Compute sentiment vector for the predicted keywords
            pred_vec = [
                1.0 * analyzer.polarity_scores(kw)["pos"] + 0.5 * analyzer.polarity_scores(kw)["neu"]
                for kw in sampled_pred
            ]
            pred_vec = np.array(pred_vec).reshape(1, -1)

            # Compute and clamp cosine similarity
            sim = cosine_similarity(gt_vec, pred_vec)[0][0]
            review_sims.append(max(0.0, min(1.0, sim)))

        # Average similarity for this review
        similarities.append(np.mean(review_sims))

    if not similarities:
        return None

    return round(np.mean(similarities), 4)


### Sentiment-Aware Evaluation

In this section, we evaluate the sentiment alignment of each model’s predicted keywords using two complementary metrics: **Average Sentiment Similarity (ASS)** and **Review-wise Sampled Sentiment Similarity (RSSS)**.

These metrics are designed to assess whether the keywords extracted by each model reflect the **emotional tone** of the movie they describe. While keyword extraction models typically focus on semantic relevance, our goal is to understand how well they capture **affective meaning**.

- **ASS** measures the **global emotional similarity** between the predicted keywords and a sentiment reference derived from the ground truth. It provides a scalar indicator of how closely, on average, the extracted keywords align with the expected emotional tone.

- **RSSS** performs a more **fine-grained, review-level evaluation**, comparing the sentiment of predicted keywords to that of sampled ground truth keywords. By introducing random sampling and averaging over reviews, this metric highlights whether the model consistently produces emotionally coherent predictions across different user opinions.

Higher values in both metrics suggest that the model is more **emotionally aligned** with the content, favoring keywords that not only describe the topic but also convey its **affective intensity**.

The results are summarized in the following table to enable **global comparison across models**.

In [24]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Ground Truth
gt_keywords = kw_ground_truth["Keyword"].tolist()
gt_keywords = [kw for kw in gt_keywords if isinstance(kw, str)]
gt_keywords = list(gt_keywords)  # Optional deduplication

# Compute reference sentiment from GT keywords
gt_sentiments = []
for kw in gt_keywords:
    score = analyzer.polarity_scores(kw)
    sentiment = 1.0 * score["pos"] + 0.5 * score["neu"]
    gt_sentiments.append(sentiment)

gt_sentiment_value = np.mean(gt_sentiments) if gt_sentiments else 0.5

# Models and predicted keyword collection
models_to_evaluate = ["base", "sentiment"]
all_predictions = {model: set() for model in models_to_evaluate}
all_sentiments_per_model = {model: [] for model in models_to_evaluate}

for _, row in selected_film.iterrows():
    for model in models_to_evaluate:
        pred_col = f"keywords_{model}"
        review_sentiments = []

        if pred_col in row and isinstance(row[pred_col], list):
            for kw, _ in row[pred_col]:
                if isinstance(kw, str):
                    # Add keyword to global set
                    all_predictions[model].add(kw)

                    # Compute sentiment score
                    score = analyzer.polarity_scores(kw)
                    sentiment = 1.0 * score["pos"] + 0.5 * score["neu"]
                    review_sentiments.append(sentiment)

        if len(review_sentiments) > 1:
            all_sentiments_per_model[model].append(review_sentiments)

# Evaluation
sentiment_scores = []

for model in models_to_evaluate:
    pred_keywords = list(all_predictions[model])

    # ASS: global sentiment similarity with average GT sentiment
    ass = compute_average_sentiment_similarity(all_sentiments_per_model[model], gt_sentiment_value)

    # RSSS: review-wise cosine similarity with sampled GT
    rsss = compute_reviewwise_sentiment_similarity(
        selected_film,
        gt_keywords=gt_keywords,
        pred_col=f"keywords_{model}",
        analyzer=analyzer,
        k_samples=20,
        seed=42
    )

    sentiment_scores.append({
        "Model": model,
        "Avg Sentiment Similarity": round(ass, 4) if ass is not None else None,
        "Review-wise Sampled Similarity": round(rsss, 4) if rsss is not None else None
    })

# Output as DataFrame
sentiment_df = pd.DataFrame(sentiment_scores).set_index("Model")
sentiment_df.style.format(precision=4).set_caption("Global Sentiment-Aware Evaluation Summary")


,Avg Sentiment Similarity,Review-wise Sampled Similarity
Model,,
base,0.9485,0.9214
sentiment,0.9621,0.9344


## Evaluation Across All Movies

This section automatically processes all `.pkl` files in the `Extracted_Keywords` directory, where each file corresponds to a single movie and contains predicted keywords generated by different models.

For **each movie**:

- The corresponding **ground truth keywords** are loaded.
- Predicted keywords from both models — **Base** and **Sentiment-aware** — are aggregated across all reviews.
- The following **global evaluation metrics** are computed:

#### Unweighted Metrics
- **Precision**, **Recall**, and **F1-score**  
  Based on approximate string matching between predicted and ground truth keywords, without considering prediction scores.

#### Score-aware Metrics
- **Weighted Precision**, **Weighted Recall**, **Weighted F1-score**  
  Evaluate prediction correctness while incorporating confidence scores.
  
- **nDCG@5**  
  Measures the ranking quality of the top 5 predicted keywords, rewarding correct keywords ranked higher.

#### Semantic Metrics
- **Semantic Precision**, **Semantic Recall**, **Semantic F1-score**  
  Computed using cosine similarity between **sentence embeddings** of predicted and reference keywords.

#### Sentiment Alignment Metrics
- **Average Sentiment Similarity (ASS)**  
  Measures the global emotional alignment between predicted keywords and the average sentiment of the ground truth keywords.

- **Review-wise Sampled Sentiment Similarity (RSSS)**  
  Computes a robust, review-level cosine similarity between the sentiment of sampled predicted keywords and sampled ground truth keywords, averaged across reviews.

All metrics are computed **globally per movie**, not per review, and the results are compiled into a **comprehensive summary table** for comparison across models.


In [18]:
# Paths
keywords_dir = "../Dataset/Extracted_Keywords/"
ground_truth_path = "../Dataset/keywords_ground_truth.pkl"

# Load ground truth keywords
keywords_ground_truth = pd.read_pickle(ground_truth_path)

# Models to evaluate
models_to_evaluate = ["base", "sentiment"]

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Container to store results for all movies and models
global_results = []

# Iterate over all predicted keyword files (one file per movie)
for file in os.listdir(keywords_dir):
    if file.endswith(".pkl") and file.startswith("kw_"):
        movie_name = file.replace("kw_", "").replace(".pkl", "")
        file_path = os.path.join(keywords_dir, file)

        try:
            # Load predicted keywords DataFrame and movie ID
            selected_film = pd.read_pickle(file_path)
            selected_film_id = selected_film["Movie_ID"].iloc[0]

            # Get ground truth keywords for this movie
            kw_ground_truth = keywords_ground_truth[
                keywords_ground_truth["Movie_ID"] == selected_film_id
            ]
            gt_keywords = kw_ground_truth["Keyword"].dropna().tolist()

            # Compute average sentiment of ground truth keywords
            gt_sentiments = []
            for kw in gt_keywords:
                scores = analyzer.polarity_scores(kw)
                sentiment = 1.0 * scores["pos"] + 0.5 * scores["neu"]
                gt_sentiments.append(sentiment)
            sentiment_gt = sum(gt_sentiments) / len(gt_sentiments) if gt_sentiments else 0.5

            # Compute average sentiment from all review texts (used only if needed)
            review_texts = selected_film["Preprocessed_Review"].dropna().tolist()
            review_sentiments = []
            for text in review_texts:
                scores = analyzer.polarity_scores(text)
                sentiment = 1.0 * scores["pos"] + 0.5 * scores["neu"]
                review_sentiments.append(sentiment)
            sentiment_text = sum(review_sentiments) / len(review_sentiments) if review_sentiments else 0.5

            # Evaluate each model globally
            for model in models_to_evaluate:
                pred_col = f"keywords_{model}"

                # Lists of lists for evaluation functions:
                # - pred_kw_per_review: list of lists of keywords (strings) for evaluate_keywords()
                # - pred_kwscore_per_review: list of lists of (keyword, score) tuples for evaluate_keywords_weighted()
                pred_kw_per_review = []
                pred_kwscore_per_review = []
                all_sentiments_per_review = []

                # Iterate over reviews to build these lists
                for _, row in selected_film.iterrows():
                    if pred_col in row and isinstance(row[pred_col], list):

                        # Extract keywords only for evaluate_keywords()
                        pred_kw_only = [kw for kw, _ in row[pred_col] if isinstance(kw, str)]

                        # Extract (keyword, score) tuples for evaluate_keywords_weighted()
                        pred_kw_score = [
                            (kw, score) for kw, score in row[pred_col]
                            if isinstance(kw, str) and isinstance(score, (float, int))
                        ]

                        # Append per review keyword lists if not empty
                        if pred_kw_only:
                            pred_kw_per_review.append(pred_kw_only)
                        if pred_kw_score:
                            pred_kwscore_per_review.append(pred_kw_score)

                        # Compute sentiment of predicted keywords for sentiment alignment
                        review_sentiments = []
                        for kw, _ in pred_kw_score:
                            scores = analyzer.polarity_scores(kw)
                            sentiment = 1.0 * scores["pos"] + 0.5 * scores["neu"]
                            review_sentiments.append(sentiment)
                        if review_sentiments:
                            all_sentiments_per_review.append(review_sentiments)

                # Compute classic precision, recall, F1
                precision, recall, f1 = evaluate_keywords(pred_kw_per_review, gt_keywords)

                # Compute weighted precision, recall, F1
                w_precision, w_recall, w_f1 = evaluate_keywords_weighted(pred_kwscore_per_review, gt_keywords)

                # Compute semantic precision, recall, F1
                flat_kw_list = [kw for review in pred_kw_per_review for kw in review]
                flat_kw_list = list(set(flat_kw_list))
                semantic_precision, semantic_recall, semantic_f1 = evaluate_semantic_keywords(
                    flat_kw_list, gt_keywords, device=device, threshold=0.65
                )

                # Compute Avg Sentiment Similarity
                ass = compute_average_sentiment_similarity(all_sentiments_per_review, sentiment_gt)

                # Compute Review-wise Sampled Sentiment Similarity (Top-5 sampling)
                rsss = compute_reviewwise_sentiment_similarity(
                    selected_film, gt_keywords, pred_col, analyzer, k_samples=20, seed=42
                )

                # Store results for this movie-model pair
                global_results.append({
                    "Movie": movie_name,
                    "Model": model,
                    "Precision": round(precision, 4),
                    "Recall": round(recall, 4),
                    "F1-score": round(f1, 4),
                    "Weighted Precision": round(w_precision, 4),
                    "Weighted Recall": round(w_recall, 4),
                    "Weighted F1-score": round(w_f1, 4),
                    "Semantic Precision": round(semantic_precision, 4),
                    "Semantic Recall": round(semantic_recall, 4),
                    "Semantic F1-score": round(semantic_f1, 4),
                    "Avg Sentiment Similarity": round(ass, 4) if ass is not None else None,
                    "Review-wise Sampled Similarity": round(rsss, 4) if rsss is not None else None
                })

        except Exception as e:
            print(f"Error processing {file}: {e}")

# Create final DataFrame and sort results
final_df = pd.DataFrame(global_results)
final_df = final_df.sort_values(by=["Movie", "Model"]).reset_index(drop=True)
final_df.style.format(precision=4).set_caption("Global Evaluation Summary per Movie and Model")

,Movie,Model,Precision,Recall,F1-score,Weighted Precision,Weighted Recall,Weighted F1-score,Semantic Precision,Semantic Recall,Semantic F1-score,Avg Sentiment Similarity,Review-wise Sampled Similarity
0,GoodBadUgly,base,0.9341,0.3612,0.5210,0.9352,0.4857,0.6394,0.8691,0.7888,0.8270,0.9321,0.6641
1,GoodBadUgly,sentiment,0.9074,0.3391,0.4937,0.9104,0.4481,0.6006,0.7477,0.8043,0.7750,0.9600,0.6825
2,HarryPotter,base,0.9194,0.3394,0.4957,0.9202,0.4552,0.6091,0.9016,0.8820,0.8917,0.9694,0.8925
3,HarryPotter,sentiment,0.9023,0.3251,0.4779,0.9043,0.4348,0.5873,0.8966,0.8976,0.8971,0.9623,0.8867
4,IndianaJones,base,0.9125,0.3678,0.5243,0.9176,0.4902,0.6390,0.9628,0.7978,0.8726,0.9485,0.9214
5,IndianaJones,sentiment,0.8805,0.3385,0.4890,0.8955,0.4489,0.5981,0.9616,0.8162,0.8830,0.9621,0.9344
6,LaLaLand,base,0.8917,0.3193,0.4702,0.9063,0.4422,0.5943,0.8537,0.7759,0.8129,0.9526,0.9526
7,LaLaLand,sentiment,0.8928,0.3126,0.4631,0.8962,0.4212,0.5731,0.7331,0.7897,0.7604,0.9459,0.9463
8,Oppenheimer,base,0.9384,0.3207,0.4781,0.9470,0.4611,0.6202,0.9905,0.8556,0.9181,0.9492,0.9492
9,Oppenheimer,sentiment,0.9247,0.3090,0.4632,0.9219,0.4328,0.5891,0.8451,0.9428,0.8913,0.9458,0.9456


In [23]:
# Save the df 
final_df.to_csv("base_vs_sentiment_evaluation.csv", index=False)